# Deformation map

This notebook will show how to compute a deformation map, i.e. $F_\text{calc}^\text{TAAM} - F_\text{calc}^\text{IAM}$.
We first load the structure information from a .cif file or a .pdb file. Here, a molecule in a P1 box is provided, but this can be replaced with whatever structure you have
Next, we initialize an IAM calculator and a TAAM calculator, set the resolution we want, and compute a deformation map. This is then saved in an .mtz-file.

In [1]:
import pydiscamb
from cctbx import miller
from iotbx import pdb
from pathlib import Path

# Replace this with a path to your structure file
structure_file = Path().resolve() / "data" / "tyrosine.pdb"

# Load structure
xrs = pdb.input(file_name=str(structure_file)).xray_structure_simple()
xrs.scattering_type_registry(table="xray")

# Prepare calculators
iam_calculator = pydiscamb.DiscambWrapper(
    xrs,
    method=pydiscamb.FCalcMethod.IAM,
)
taam_calculator = pydiscamb.DiscambWrapper(
    xrs,
    method=pydiscamb.FCalcMethod.TAAM,
)

# Set resolution
miller_set = miller.build_set(
    crystal_symmetry=xrs.crystal_symmetry(), 
    anomalous_flag=False, 
    d_min=0.7,
)
iam_calculator.set_indices(miller_set.indices())
taam_calculator.set_indices(miller_set.indices())

# Compute structure factors
fcalc_iam = iam_calculator.f_calc()
fcalc_taam = taam_calculator.f_calc()

# Compute deformation map
deformation_map = fcalc_taam - fcalc_iam

# Create a miller.array to save
deformation_map_array = miller_set.array(
    data=deformation_map,
)
deformation_map_array.write_mtz(
    file_name="deformation_map.mtz",
)

{"algorithm":"macromol","assignment csv":"/tmp/pyDiSCaMB-atom-assignmentva_767g_.csv","bank path":"/home/vife5188/Phenix/phenix-2.0-5730/lib/python3.9/site-packages/pydiscamb/data/MATTS2021databank.txt","electron scattering":false,"model":"taam","table":"IT92"}


Below is a capture from Coot, showing the resulting deformation map at 3.0 RMSD

![tyrosine_difference_map.png](./data/tyrosine_difference_map.png)
